# Importación de librerias

In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import Normalizer

# Importación de datos

join: tabla de prestamos junto a la información de los ítems después de la limpieza y extracción de atributos adicionales. 

In [2]:
#importamos la tabla de join 
join = pd.read_json('https://www.dropbox.com/s/i1komhf7u1c4y95/joinTablas.json?dl=1')


In [3]:
#Eliminamos algunas columnas que no nos interesan para este notebook
join = join.drop(["Fecha","Dewey","Facultad","Temas","Union","TipoItem"], axis=1)
#Ejemplos y columnas del dataframe
display(join.head(3))
display(join.columns)

,RowID,IDItem,NumeroUbicacion,Ubicacion,Llave,Programa,IDUsuario,Year,Signatura,FechaCreacion,Autor,Titulo,AnioPublicacion,DeweyEspecifico,TemaDewey,DeweyUnidad,DeweyDecena,DeweyCentena
0,Row0,80000005327627,720.98 A71S,COL-GRAL-2,866245,CARRERA DE ARQUITECTURA,c361c772cd0220bb16dcfb2ad803e4903334ab94,2019,720.98,2013,"Arango Cardinal, Silvia 1948-",Ciudad y arquitectura seis generaciones que co...,2012.0,720.98,Arquitectura latinoamericana,720,720,700
1,Row1,80000001191496,720.9861 A71,COL-GRAL-2,309945,CARRERA DE ARQUITECTURA,c361c772cd0220bb16dcfb2ad803e4903334ab94,2019,720.9861,1993,"Arango Cardinal, Silvia 1948-",Historia de la arquitectura en Colombia Silvia...,1993.0,720.9861,Arquitectura colombiana,720,720,700
2,Row2,80000004979759,540 CH15Q 2010,COL-GRAL-3,822727,CARRERA DE ARQUITECTURA,87b0e5a61ed712ddfaf5d478ad68c87c825997e9,2019,540,2011,"Chang, Raymond",Química Raymond Chang ; revisión técnica Rosa ...,2010.0,540,Química,540,540,500


Index(['RowID', 'IDItem', 'NumeroUbicacion', 'Ubicacion', 'Llave', 'Programa',
       'IDUsuario', 'Year', 'Signatura', 'FechaCreacion', 'Autor', 'Titulo',
       'AnioPublicacion', 'DeweyEspecifico', 'TemaDewey', 'DeweyUnidad',
       'DeweyDecena', 'DeweyCentena'],
      dtype='object')

# Preparación de los datos
Crearemos la columna de pesos así como el dataframe que identificará los gustos y preferencias de los usuarios. Estos dataframes se crearán teniendo en cuenta 3 columnas diferentes:
* DeweyUnidad 
* DeweyDecena
* DeweyCentena


**Creación de columna "Peso"**:
A partir del análisis con expertos en negocio, se ha entendido que es importante tomar en cuenta el cambio de gustos de los usuarios a través del tiempo y dar menos peso a prestamos que se realizaron en el pasado a prestamos más recientes. Para esto se creará una nueva columna denominada "Peso" la cual determinará el peso quue tiene dicho prestamo.
La disminución del peso será exponencial según la diferencia con el año actual y se calculará con la siguiente formula:

$Peso = \frac{1}{2^{(AñoActual-20xx)}}$


In [4]:
#Obtenemos el año actual
import datetime
now = datetime.datetime.now()
anio_actual = int(now.year)
print("Anio Actual: ", anio_actual)

Anio Actual:  2021


In [5]:
#Creamos la columna pesos para el dataframe a partir del año en que se realizó el prestamo
join["Peso"] = join.apply(lambda row: 1/2**(anio_actual-row.Year), axis=1 )
join[["Year","Peso"]]

,Year,Peso
0,2019,0.250
1,2019,0.250
2,2019,0.250
3,2019,0.250
4,2019,0.250
...,...,...
489732,2018,0.125
489733,2018,0.125
489734,2018,0.125
489735,2018,0.125


La siguiente función crea la tabla de preferencias y gustos de los usuarios a partir de los prestamos y la fecha de estos(Columna Peso).

Cada Fila representa un usuario, las columnas son los deweys y los datos es el nivel de pertenencia del usuario frente a dicho dewey.

In [6]:
agrupacion = join.groupby(["IDUsuario","DeweyUnidad"])["Peso"].sum().reset_index(name="Peso")
agrupacion.head(5)

,IDUsuario,DeweyUnidad,Peso
0,00063d52cf68c65d2a569e95c40345c4a305ccc7,658,0.500
1,0006b3bdabeaa5389b211c8487bd67d339c97ef7,378,0.125
2,0006b3bdabeaa5389b211c8487bd67d339c97ef7,657,0.375
3,000a0630c32a437052696f6921e3181f155291d0,1,0.500
4,000c040064aedd76057c5309064a3823db970644,657,0.125


In [7]:
#crearTablaPesos: crea la tabla de pesos por usuario
#parámetros
    #columna = {deweyUnidad|deweyDecena|deweyCentena}
def crearTablaPesos(columna):
    #Tabla = pesos_usuarios
        #columnas = deweys
        #filas = usuarios
        #dato = peso que tiene el usuario en dicho dewey
    #creamos las columnas a partir de los deweys diferentes. 
    agrupacion = join.groupby(["IDUsuario",columna])["Peso"].sum().reset_index(name="Peso")
    display(agrupacion.head(5))
    
    #cración del dataframe
    pesos_usuarios = pd.DataFrame()
    #Dataframe auxiliar para acelerar el proceso de concat
    aux = pd.DataFrame()
    
    ids = agrupacion["IDUsuario"].unique()
    print("Total de IDs de usuarios: ", len(ids))
    #Recorremos cada uno de los usuarios
    for usuario in ids:
        #obtenemos todos los prestamos del usuario
        prestamos = agrupacion.loc[agrupacion["IDUsuario"]==usuario]
        columnas = prestamos[columna].values
        pesos = prestamos["Peso"].values
        fila = pd.DataFrame(data = [pesos], columns = columnas)
        fila["IDUsuario"] = usuario
        aux = pd.concat([aux,fila])
        #cada 100 registros concatenamos al dataframe general
        if(aux.shape[0] == 100):
            pesos_usuarios = pd.concat([pesos_usuarios,aux])
            aux = pd.DataFrame()
    pesos_usuarios = pd.concat([pesos_usuarios,aux])
    #Cambiamos los nil por 0
    pesos_usuarios = pesos_usuarios.fillna(0)
    pesos_usuarios.reset_index(drop=True, inplace=True)
    return pesos_usuarios

Se crean 3 tablas de peso diferentes según la columna que se tuvo en cuenta:

{deweyUnidad|deweyDecena|deweyCentena}

In [8]:
pesos_usuarios_unidad=crearTablaPesos("DeweyUnidad")
display(pesos_usuarios_unidad.head(5))
print("Forma de la tabla: ", pesos_usuarios_unidad.shape)

,IDUsuario,DeweyUnidad,Peso
0,00063d52cf68c65d2a569e95c40345c4a305ccc7,658,0.500
1,0006b3bdabeaa5389b211c8487bd67d339c97ef7,378,0.125
2,0006b3bdabeaa5389b211c8487bd67d339c97ef7,657,0.375
3,000a0630c32a437052696f6921e3181f155291d0,1,0.500
4,000c040064aedd76057c5309064a3823db970644,657,0.125


Total de IDs de usuarios:  24407


,658,IDUsuario,378,657,1,227,347,350,808,812,...,964,367,383,494,413,496,799,855,876,97
0,0.5,00063d52cf68c65d2a569e95c40345c4a305ccc7,0.000,0.000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0006b3bdabeaa5389b211c8487bd67d339c97ef7,0.125,0.375,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,000a0630c32a437052696f6921e3181f155291d0,0.000,0.000,0.5,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,000c040064aedd76057c5309064a3823db970644,0.000,0.125,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,000f23e62e6d4995ede7090c6447ad6736edc4de,0.000,0.000,0.0,0.5,0.5,0.5,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Forma de la tabla:  (24407, 756)


In [9]:
pesos_usuarios_decena = crearTablaPesos("DeweyDecena")
display(pesos_usuarios_decena.head(5))
print("Forma de la tabla: ", pesos_usuarios_decena.shape)

,IDUsuario,DeweyDecena,Peso
0,00063d52cf68c65d2a569e95c40345c4a305ccc7,650,0.500
1,0006b3bdabeaa5389b211c8487bd67d339c97ef7,370,0.125
2,0006b3bdabeaa5389b211c8487bd67d339c97ef7,650,0.375
3,000a0630c32a437052696f6921e3181f155291d0,0,0.500
4,000c040064aedd76057c5309064a3823db970644,650,0.125


Total de IDs de usuarios:  24407


,650,IDUsuario,370,0,220,340,350,800,810,20,...,560,640,470,310,10,480,90,990,50,30
0,0.500,00063d52cf68c65d2a569e95c40345c4a305ccc7,0.000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.375,0006b3bdabeaa5389b211c8487bd67d339c97ef7,0.125,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.000,000a0630c32a437052696f6921e3181f155291d0,0.000,0.5,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.125,000c040064aedd76057c5309064a3823db970644,0.000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.000,000f23e62e6d4995ede7090c6447ad6736edc4de,0.000,0.0,0.5,0.5,0.5,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Forma de la tabla:  (24407, 100)


In [10]:
pesos_usuarios_centena = crearTablaPesos("DeweyCentena")
display(pesos_usuarios_centena.head(5))
print("Forma de la tabla: ", pesos_usuarios_centena.shape)

,IDUsuario,DeweyCentena,Peso
0,00063d52cf68c65d2a569e95c40345c4a305ccc7,600,0.500
1,0006b3bdabeaa5389b211c8487bd67d339c97ef7,300,0.125
2,0006b3bdabeaa5389b211c8487bd67d339c97ef7,600,0.375
3,000a0630c32a437052696f6921e3181f155291d0,0,0.500
4,000c040064aedd76057c5309064a3823db970644,600,0.125


Total de IDs de usuarios:  24407


,600,IDUsuario,300,0,200,800,700,100,400,500,900,-900
0,0.500,00063d52cf68c65d2a569e95c40345c4a305ccc7,0.000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.375,0006b3bdabeaa5389b211c8487bd67d339c97ef7,0.125,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.000,000a0630c32a437052696f6921e3181f155291d0,0.000,0.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.125,000c040064aedd76057c5309064a3823db970644,0.000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.000,000f23e62e6d4995ede7090c6447ad6736edc4de,1.000,0.0,0.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Forma de la tabla:  (24407, 12)


Exportamos e Importamos las tablas para agilizar el proceso

In [13]:
pesos_usuarios_unidad.to_json(r'C:\Users\user\Downloads\pesos_usuario_x_dewey_unidad.json')
pesos_usuarios_decena.to_json(r'C:\Users\user\Downloads\pesos_usuario_x_dewey_decena.json')
pesos_usuarios_centena.to_json(r'C:\Users\user\Downloads\pesos_usuario_x_dewey_centena.json')

In [11]:
pesos_usuarios_unidad = pd.read_json('https://www.dropbox.com/s/aitygqwn9q47rlg/pesos_usuario_x_dewey_unidad.json?dl=1')

In [12]:
pesos_usuarios_decena = pd.read_json('https://www.dropbox.com/s/vr6ehn8xjhojuba/pesos_usuario_x_dewey_decena.json?dl=1')

In [13]:
pesos_usuarios_centena = pd.read_json('https://www.dropbox.com/s/2vnntgjnqpijkgg/pesos_usuario_x_dewey_centena.json?dl=1')

# Normalización de pesos
En busca de que dos usuarios con mismos gustos pero diferente cantidad de prestamos sean equivalentes se decidió  por normalizar cada una de las filas. Así un usuario con 3 prestamos en el dewey 600 y otro con un único prestamo en el dewey 600 se velven equivalente al tener gustos únicamente por este dewey.

In [14]:
#función: normalizar las filas por valores entre cero y uno.
    #Parámetros: pesos_usuario, representa la matriz dispersa que se va a normalizar.
    #{pesos_usuarios_unidad|pesos_usuarios_decena|pesos_usuarios_centena}
def normalizar_pesos(pesos_usuario):
    print("Normalizando pesos...Iniciando")
    usuarios = pesos_usuario['IDUsuario']
    pesos_usuario = pesos_usuario.apply(pd.to_numeric, errors='coerce').drop(["IDUsuario"],axis=1)
    #display(pesos_usuario)
    #Normalizamos por fila
    sumatoria = pesos_usuario.max(axis=1)
    pesos_norm = pesos_usuario.div(pesos_usuario.sum(axis=1), axis=0)
    pesos_norm_id = pesos_norm.copy()
    # a la tabla le agregamos la columna de IDUsuario para poder identificar que pesos son de cada usuario
    if len(pesos_norm_id) == len(join.IDUsuario.unique()):
        pesos_norm_id['IDUsuario'] = usuarios
    print("Normalizando pesos...Acabado")
    return pesos_norm_id

In [15]:
pesos_norm_id_unidad = normalizar_pesos(pesos_usuarios_unidad)
display(pesos_norm_id_unidad.shape)
pesos_norm_id_unidad.head(3)

Normalizando pesos...Iniciando
Normalizando pesos...Acabado


(24407, 756)

,658,378,657,1,227,347,350,808,812,813,...,367,383,494,413,496,799,855,876,97,IDUsuario
0,1.0,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,00063d52cf68c65d2a569e95c40345c4a305ccc7
1,0.0,0.25,0.75,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0006b3bdabeaa5389b211c8487bd67d339c97ef7
2,0.0,0.00,0.00,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,000a0630c32a437052696f6921e3181f155291d0


In [16]:
pesos_norm_id_decena = normalizar_pesos(pesos_usuarios_decena)
display(pesos_norm_id_decena.shape)
pesos_norm_id_decena.head(3)

Normalizando pesos...Iniciando
Normalizando pesos...Acabado


(24407, 100)

,650,370,0,220,340,350,800,810,20,330,...,640,470,310,10,480,90,990,50,30,IDUsuario
0,1.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,00063d52cf68c65d2a569e95c40345c4a305ccc7
1,0.75,0.25,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0006b3bdabeaa5389b211c8487bd67d339c97ef7
2,0.00,0.00,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,000a0630c32a437052696f6921e3181f155291d0


In [17]:
pesos_norm_id_centena = normalizar_pesos(pesos_usuarios_centena)
display(pesos_norm_id_centena.shape)
pesos_norm_id_centena.head(3)

Normalizando pesos...Iniciando
Normalizando pesos...Acabado


(24407, 12)

,600,300,0,200,800,700,100,400,500,900,-900,IDUsuario
0,1.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,00063d52cf68c65d2a569e95c40345c4a305ccc7
1,0.75,0.25,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0006b3bdabeaa5389b211c8487bd67d339c97ef7
2,0.00,0.00,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,000a0630c32a437052696f6921e3181f155291d0


In [18]:
pesos_norm_id_unidad.to_json(r'C:\Users\user\Downloads\pesos_norm_id_unidad.json')


# Agrupación disfusa
Para la agrupación cada usuario podrá pertenecer a uno o mas deweys. Cada usuario tendra un grado de pertenencia a un dewey en especifico dependiendo de sus gustos y preferencias resumidos en las matrices dispersas. 
La función se debe recorrer mínimo 3 veces con diferentes matrices de dispersión:
* pesos_usuarios_unidad
* pesos_usuarios_decena
* pesos_usuarios_centena

In [19]:
#Elimina los valores diferentes de cero para una fila especifica
#Sirve para visualizar únicamente los deweys donde el usuario tiene prestamos. 
def eliminar_cero(id_usuario, df_pesos):
    m1 = (df_pesos['IDUsuario'] == id_usuario)
    m2 = (df_pesos[m1] != 0).all()
    return df_pesos.loc[m1,m2]

In [20]:
eliminar_cero("33c7a2220802831409c62333e13f068363a5d768", pesos_norm_id_unidad)

,1,517,5,IDUsuario
4945,0.333333,0.333333,0.333333,33c7a2220802831409c62333e13f068363a5d768


In [21]:
#Esta función asocia una grado de pertenencia
#Parámetros:
    #pesos_usuarios: matrix dispersa de pesos {pesos_usuarios_unidad|pesos_usuarios_decena|pesos_usuarios_centena}
    #nombre_archivo: nombre del archivo a exportar con la tabla de df_pertenencia
#se crea la tabla df_pertenencia: describe la pertenecia de cada usuario a un cluster(dewey)
def clustering(pesos_usuarios):
    print("Comenzando clustering...")
    #normalizamos los pesos
    pesos_norm_id_unidad = normalizar_pesos(pesos_usuarios)
    #ponemos el usuario a ambos dataframes
    pesos_usuarios["IDUsuario"] = pesos_norm_id_unidad["IDUsuario"]
    #Creamos la estructura del dataframe
    df_pertenencia = pd.DataFrame(columns = ['IDUsuario', 'Cluster', 'Pertenencia','Peso'])
    #iteramos por cada fila(usuario) del dataframe
    for index, row in pesos_norm_id_unidad.iterrows():
        #Extraemos los usuarios
        usuario = row["IDUsuario"]
        #Extraemos los pesos deiferentes de cero normalizados y sin normalizar
        usuario_limpio_norm = eliminar_cero(row["IDUsuario"], pesos_norm_id_unidad)
        usuario_limpio_orig = eliminar_cero(row["IDUsuario"], pesos_usuarios)
        pertenecias = usuario_limpio_norm.drop(["IDUsuario"], axis=1).values
        pesos = usuario_limpio_orig.drop(["IDUsuario"], axis=1).values
        #los clusters son las columnas de estos dataframes
        clusters = usuario_limpio_norm.drop(["IDUsuario"], axis=1).columns
        #creamos un array que tenga las veces necesarias al usuario
        tamanio = len(clusters)
        usuarios = np.repeat([usuario], tamanio)
        #agregamos la fila al dataframe
        fila = pd.DataFrame(data = {'IDUsuario': usuarios, 'Cluster': clusters
                                   , 'Pertenencia': pertenecias[0], 'Peso': pesos[0]})
        df_pertenencia = df_pertenencia.append(fila)
    print("Finalizando clustering...")
    return df_pertenencia

In [22]:
pesos_clustering_unidad = clustering(pesos_usuarios_unidad)
display(pesos_clustering_unidad.head(3))

Comenzando clustering...
Normalizando pesos...Iniciando
Normalizando pesos...Acabado
Finalizando clustering...


,IDUsuario,Cluster,Pertenencia,Peso
0,00063d52cf68c65d2a569e95c40345c4a305ccc7,658,1.00,0.500
0,0006b3bdabeaa5389b211c8487bd67d339c97ef7,378,0.25,0.125
1,0006b3bdabeaa5389b211c8487bd67d339c97ef7,657,0.75,0.375


In [23]:
pesos_clustering_decena = clustering(pesos_usuarios_decena)
display(pesos_clustering_decena.head(3))

Comenzando clustering...
Normalizando pesos...Iniciando
Normalizando pesos...Acabado
Finalizando clustering...


,IDUsuario,Cluster,Pertenencia,Peso
0,00063d52cf68c65d2a569e95c40345c4a305ccc7,650,1.00,0.500
0,0006b3bdabeaa5389b211c8487bd67d339c97ef7,650,0.75,0.375
1,0006b3bdabeaa5389b211c8487bd67d339c97ef7,370,0.25,0.125


In [24]:
pesos_clustering_centena = clustering(pesos_usuarios_centena)
display(pesos_clustering_centena.head(3))

Comenzando clustering...
Normalizando pesos...Iniciando
Normalizando pesos...Acabado
Finalizando clustering...


,IDUsuario,Cluster,Pertenencia,Peso
0,00063d52cf68c65d2a569e95c40345c4a305ccc7,600,1.00,0.500
0,0006b3bdabeaa5389b211c8487bd67d339c97ef7,600,0.75,0.375
1,0006b3bdabeaa5389b211c8487bd67d339c97ef7,300,0.25,0.125


In [26]:
pesos_clustering_unidad.reset_index(drop=True, inplace=True)
pesos_clustering_decena.reset_index(drop=True, inplace=True)
pesos_clustering_centena.reset_index(drop=True, inplace=True)

Exportamos e importamos los datos para agilizar el proceso

In [1]:
pesos_clustering_unidad.to_json(r'C:\Users\user\Downloads\pesos_clustering_unidad.json')
pesos_clustering_decena.to_json(r'C:\Users\user\Downloads\pesos_clustering_decena.json')
pesos_clustering_centena.to_json(r'C:\Users\user\Downloads\pesos_clustering_centena.json')

NameError: name 'pesos_clustering_unidad' is not defined

In [25]:
pesos_clustering_unidad = pd.read_json('https://www.dropbox.com/s/6j30n8y3fn8358l/pesos_clustering_unidad.json?dl=1')
pesos_clustering_decena = pd.read_json('https://www.dropbox.com/s/6m7vbpfq8b8qz4s/pesos_clustering_decena.json?dl=1')
pesos_clustering_centena = pd.read_json('https://www.dropbox.com/s/3rjqco5swu55cna/pesos_clustering_centena.json?dl=1')

In [27]:
display(pesos_clustering_unidad.head(3))
print(pesos_clustering_unidad.shape)

,IDUsuario,Cluster,Pertenencia,Peso
0,00063d52cf68c65d2a569e95c40345c4a305ccc7,658,1.00,0.500
1,0006b3bdabeaa5389b211c8487bd67d339c97ef7,378,0.25,0.125
2,0006b3bdabeaa5389b211c8487bd67d339c97ef7,657,0.75,0.375


(125653, 4)
